In [1]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/official/automl/automl-text-classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/official/automl/automl-text-classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/ai-platform-samples/raw/master/ai-platform-unified/notebooks/official/automl/automl-text-classification.ipynb">
      Open in Google Cloud Notebooks
    </a>
  </td>    
</table>

# Vertex AI: Create, train, and deploy an AutoML text classification model

## Overview

This notebook walks you through the major phases of building and using a text classification model on [Vertex AI](https://cloud.google.com/vertex-ai/docs/). In this notebook, you use the "Happy Moments" sample dataset to train a model. The resulting model classifies happy moments into categores that reflect the causes of happiness. 

### Objective

In this notebook, you learn how to:

* Set up your development environment
* Create a dataset and import data
* Train an AutoML model
* Get and review evaluations for the model
* Deploy a model to an endpoint
* Get online predictions
* Get batch predictions

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI Training and Serving
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage

## Before you begin

**Note:** This notebook does not require a GPU runtime.

### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI, Cloud Storage, and Compute Engine APIs](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com). 

1. Follow the "**Configuring your project**" instructions from the Vertex Pipelines documentation.

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

### Install additional packages

This notebook uses the Python SDK for Vertex AI, which is contained in the `python-aiplatform` package. You must first install the package into your development environment.

In [6]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform google-cloud-storage jsonlines

### Set your project ID

Finally, you must initialize the client library before you can send requests to the Vertex AI service. With the Python SDK, you initialize the client library as shown in the following cell. Be sure to provide the ID for your Google Cloud project in the `project` variable.

This notebook uses the `us-central1` region, although you can change it to another region. 

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [7]:
from google.cloud import aiplatform
import os

PROJECT_ID = "[your-project-id]"
REGION = "us-central1"

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

aiplatform.init(project=PROJECT_ID, location=REGION)

Project ID:  erschmid-test-291318


## Create a dataset and import your data

The notebook uses the 'Happy Moments' dataset for demonstration purposes. You can change it to another text classification dataset that [conforms to the data preparation requirements](https://cloud.google.com/vertex-ai/docs/datasets/prepare-text#classification).

Using the Python SDK, you can create a dataset and import the dataset in one call to `TextDataset.create()`, as shown in the following cell.

Creating and importing data is a long-running operation. This next step can take a while. The sample waits for the operation to complete, outputting statements as the operation progresses. The statements contain the full name of the dataset that you will use in the following section.

**Note**: You can close the noteboook while you wait for this operation to complete. 

In [10]:
# Use a timestamp to ensure unique resources
from datetime import datetime
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

src_uris = "gs://cloud-ml-data/NL-classification/happiness.csv"
display_name = f"e2e-text-dataset-{TIMESTAMP}"

In [9]:
ds = aiplatform.TextDataset.create(
    display_name=display_name,
    gcs_source=src_uris,
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.single_label_classification,
    sync=True,
)

INFO:google.cloud.aiplatform.datasets.dataset:Creating TextDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TextDataset backing LRO: projects/1025771077852/locations/us-central1/datasets/2744037975301029888/operations/8748568006486917120
INFO:google.cloud.aiplatform.datasets.dataset:TextDataset created. Resource name: projects/1025771077852/locations/us-central1/datasets/2744037975301029888
INFO:google.cloud.aiplatform.datasets.dataset:To use this TextDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TextDataset('projects/1025771077852/locations/us-central1/datasets/2744037975301029888')
INFO:google.cloud.aiplatform.datasets.dataset:Importing TextDataset data: projects/1025771077852/locations/us-central1/datasets/2744037975301029888
INFO:google.cloud.aiplatform.datasets.dataset:Import TextDataset data backing LRO: projects/1025771077852/locations/us-central1/datasets/2744037975301029888/operations/7771286887347519488


KeyboardInterrupt: 

## Train your text classification model

Once your dataset has finished importing data, you are ready to train your model. To do this, you first need the full resource name of your dataset, where the full name has the format `projects/[YOUR_PROJECT]/locations/us-central1/datasets/[YOUR_DATASET_ID]`. If you don't have the resource name handy, you can list all of the datasets in your project using `TextDataset.list()`. 

As shown in the following code block, you can pass in the display name of your dataset in the call to `list()` to filter the results.


In [23]:
datasets = aiplatform.TextDataset.list(filter=f'display_name="{display_name}"')
print(datasets)

resource name: projects/1025771077852/locations/us-central1/datasets/2744037975301029888, <google.cloud.aiplatform.datasets.text_dataset.TextDataset object at 0x7f7f35f474d0> 
resource name: projects/1025771077852/locations/us-central1/datasets/7192046294771171328, <google.cloud.aiplatform.datasets.text_dataset.TextDataset object at 0x7f7f367bb110> 
resource name: projects/1025771077852/locations/us-central1/datasets/5023563074192277504, <google.cloud.aiplatform.datasets.text_dataset.TextDataset object at 0x7f7f35f46590> 
resource name: projects/1025771077852/locations/us-central1/datasets/3783876908014370816, <google.cloud.aiplatform.datasets.text_dataset.TextDataset object at 0x7f7f35f59190> 
resource name: projects/1025771077852/locations/us-central1/datasets/2672050750007279616]


Now you can begin training your model. Training the model is a two part process:

1. **Define the training job.** You must provide a display name and the type of training you want when you define the training job.
2. **Run the training job.** When you run the training job, you need to supply a reference to the dataset to use for training. At this step, you can also configure the data split percentages.

You do not need to specify [data splits](https://cloud.google.com/vertex-ai/docs/general/ml-use). The training job has a default setting of  training 80%/ testing 10%/ validate 10% if you don't provide these values.

As with importing data into the dataset, training your model can take a substantial amount of time. The client library prints out operation status messages while the training pipeline operation processes. You must wait for the training process to complete before you can get the resource name and ID of your new model, which is required for model evaluation and model deployment.

**Note**: You can close the notebook while you wait for the operation to complete.

In [24]:
# Define the training job
training_job_display_name = f"e2e-text-training-job-{TIMESTAMP}"
job = aiplatform.AutoMLTextTrainingJob(
    display_name=training_job_display_name,
    prediction_type="classification",
    multi_label=False,
)

In [27]:
# Get the dataset ID if it's not available
dataset_id = "[your-dataset-id]"

if dataset_id == "[your-dataset-id]":
    dataset = datasets[0]
    dataset_id = dataset.resource_name.split("/")[-1]
    print(f"Dataset ID: {dataset_id}")

model_display_name = f"e2e-text-classification-model-{TIMESTAMP}"

text_dataset = aiplatform.TextDataset(dataset_id)

# Run the training job
model = job.run(
    dataset=text_dataset,
    model_display_name=model_display_name,
    training_fraction_split=0.7,
    validation_fraction_split=0.2,
    test_fraction_split=0.1,
    sync=True,
)

Dataset ID: 2744037975301029888
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/830144473110413312?project=1025771077852
INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/1025771077852/locations/us-central1/trainingPipelines/830144473110413312 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/1025771077852/locations/us-central1/trainingPipelines/830144473110413312 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/1025771077852/locations/us-central1/trainingPipelines/830144473110413312 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/1025771077852/locations/us-central1/trainingPipelines/830144473110413312 current state:
PipelineState.PIPELINE_STATE_RUNNI

## Get and review model evaluation scores

After your model has finished training, you can review the evaluation scores for it.

First, you need to get the resource name of your new model. To get the resource name, list all of the models in your project. As before with datasets, you can provide filter criteria to narrow down your search.

In [31]:
models = aiplatform.Model.list(filter=f'display_name="{model_display_name}"')
print(models)

resource name: projects/1025771077852/locations/us-central1/models/8136887422844665856, <google.cloud.aiplatform.models.Model object at 0x7f7f37854850> 
resource name: projects/1025771077852/locations/us-central1/models/433313134460010496]


Using the model name (in the format `projects/[PROJECT_NAME]/locations/us-central1/models/[MODEL_ID]`), you can get its model evaluations. To get model evaluations, you must use the underlying service client.

Building a service client requires that you provide the name of the regionalized hostname used for your model. In this tutorial, the hostname is `us-central1-aiplatform.googleapis.com` because the model was created in the `us-central1` location.

In [32]:
# Get the ID of the model
model_name = "[your-model-resource-name]"
if model_name == "[your-model-resource-name]":
    model_name = models[0].resource_name
    print(f"Model name: {model_name}")


# Get a reference to the Model Service client
client_options = {"api_endpoint": "us-central1-aiplatform.googleapis.com"}
model_service_client = aiplatform.gapic.ModelServiceClient(
    client_options=client_options
)

Model name: projects/1025771077852/locations/us-central1/models/8136887422844665856
INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/1025771077852/locations/us-central1/trainingPipelines/830144473110413312 current state:
PipelineState.PIPELINE_STATE_RUNNING


Before you can view the model evaluation you must first list all of the evaluations for that model. Each model can have multiple evaluations, although new model is likely to only have one. 

In [33]:
model_evaluations = model_service_client.list_model_evaluations(parent=model_name)
model_evaluation = list(model_evaluations)[0]

Now that you have the model evaluation, you can look at your model's scores. If you have questions about what the scores mean, review the [public documentation](https://cloud.google.com/vertex-ai/docs/training/evaluating-automl-models#text).

The results returned from the service are formatted as [`google.protobuf.Value`](https://googleapis.dev/python/protobuf/latest/google/protobuf/struct_pb2.html) objects. You can transform the return object as a `dict` for easier reading and parsing.

In [34]:
from google.protobuf import json_format

model_eval_dict = json_format.MessageToDict(model_evaluation._pb)
metrics = model_eval_dict["metrics"]
confidence_metrics = metrics["confidenceMetrics"]

print(f'Area under precision-recall curve (AuPRC): {metrics["auPrc"]}')
for confidence_scores in confidence_metrics:
    metrics = confidence_scores.keys()
    print("\n")
    for metric in metrics:
        print(f"\t{metric}: {confidence_scores[metric]}")

Area under precision-recall curve (AuPRC): 0.95126826


	recall: 1.0
	precision: 0.14285715
	f1Score: 0.25
	f1ScoreAt1: 0.8812709
	precisionAt1: 0.8812709
	recallAt1: 0.8812709


	precisionAt1: 0.8812709
	f1ScoreAt1: 0.8812709
	recall: 0.9481605
	f1Score: 0.83736384
	confidenceThreshold: 0.05
	precision: 0.74975204
	recallAt1: 0.8812709


	f1Score: 0.8560999
	confidenceThreshold: 0.1
	f1ScoreAt1: 0.88122123
	recall: 0.93143815
	precisionAt1: 0.88158995
	recallAt1: 0.8808528
	precision: 0.79203695


	recall: 0.92140466
	f1ScoreAt1: 0.8811218
	confidenceThreshold: 0.15
	recallAt1: 0.88001674
	precisionAt1: 0.8822297
	f1Score: 0.8658416
	precision: 0.8165987


	recall: 0.9088629
	f1ScoreAt1: 0.8801176
	recallAt1: 0.8762542
	f1Score: 0.8692523
	confidenceThreshold: 0.2
	precision: 0.8329502
	precisionAt1: 0.8840152


	f1Score: 0.87173253
	f1ScoreAt1: 0.8806567
	precisionAt1: 0.88681644
	confidenceThreshold: 0.25
	recall: 0.89924747
	recallAt1: 0.87458193
	precision: 0.84585136


	recallAt1

## Deploy your text classification model

Once your model has completed training, you must deploy it to an _endpoint_ to get online predictions from it. When you deploy the model to an endpoint, a copy of the model is made on the endpoint with a new resource name and display name.

You can deploy multiple models to the same endpoint and split traffic between the various models assigned to the endpoint. However, you must deploy one model at a time to the endpoint. To change the traffic split percentages, you must assign new values on your second (and subsequent) models each time you deploy a new model.

The following code block demonstrates how to deploy a model. The code snippet relies on the Python SDK to create a new endpoint for deployment. During deployment, the Python SDK prints out the name of your new endpoint--you might want to record the name of the endpoint for use in online predictions.

In [ ]:
deployed_model_display_name = f"e2e-deployed-text-classification-model-{TIMESTAMP}"

model.deploy(deployed_model_display_name=deployed_model_display_name, sync=True)

INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/1025771077852/locations/us-central1/trainingPipelines/830144473110413312 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/1025771077852/locations/us-central1/trainingPipelines/6267115123253444608 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/1025771077852/locations/us-central1/trainingPipelines/830144473110413312 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/1025771077852/locations/us-central1/trainingPipelines/6267115123253444608 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/1025771077852/locations/us-central1/trainingPipelines/830144473110413312 current state:
PipelineState.PIPELINE_STATE_RUNNING
INF

In case you didn't record the name of the new endpoint, you can get a list of all your endpoints as you did before with datasets and models. For each endpoint, you can list the models deployed to that endpoint. To get a reference to the model that you just deployed, you check the `display_name` of each model deployed to the endpoint against the model you're looking for.

In [ ]:
endpoints = aiplatform.Endpoint.list()

endpoint_with_deployed_model = []

for endpoint in endpoints:
    for model in endpoint.list_models():
        if model.display_name.find(deployed_model_display_name) == 0:
            endpoint_with_deployed_model.append(endpoint)

print(endpoint_with_deployed_model)

## Get online predictions from your model

Now that you have your endpoint's resource name, you can get online predictions from the text classification model. To get the online prediction, you send a prediction request to your endpoint.

In [ ]:
endpoint_name = "[your-endpoint-name]"
if endpoint_name == "[your-endpoint-name]":
    endpoint_name = endpoints[0].resource_name
    print(f"Endpoint name: {endpoint_name}")


endpoint = aiplatform.Endpoint(endpoint_name)
content = "I got a high score on my math final!"

response = endpoint.predict(instances=[{"content": content}])

for prediction_ in response.predictions:
    ids = prediction_["ids"]
    display_names = prediction_["displayNames"]
    confidence_scores = prediction_["confidences"]
    for count, id in enumerate(ids):
        print(f"Prediction ID: {id}")
        print(f"Prediction display name: {display_names[count]}")
        print(f"Prediction confidence score: {confidence_scores[count]}")

## Get batch predictions from your model

You can get batch predictions from a text classification model without deploying it. You must first format all of your prediction instances (prediction input) in JSONL format and you must store the JSONL file in a Google Cloud Storage bucket. You must also provide a Google Cloud Storage bucket to hold your prediction output.

To start, you must first create your predictions input file in JSONL format. Each line in the JSONL document needs to be formatted like so:

```
{ "content": "gs://sourcebucket/datasets/texts/source_text.txt", "mimeType": "text/plain"}
```

The `content` field in the JSON structure must be a Google Cloud Storage URI to another document that contains the text input for prediction.
[See the documentation for more information.](https://cloud.google.com/ai-platform-unified/docs/predictions/batch-predictions#text)

In [ ]:
instances = [
    "We hiked through the woods and up the hill to the ice caves",
    "My kitten is so cute",
]
input_file_name = "batch-prediction-input.jsonl"

For batch prediction, you must supply the following:

+ All of your prediction instances as individual files on Google Cloud Storage, as TXT files for your instances
+ A JSONL file that lists the URIs of all your prediction instances
+ A Google Cloud Storage bucket to hold the output from batch prediction

For this tutorial, the following cells create a new Storage bucket, upload individual prediction instances as text files to the bucket, and then create the JSONL file with the URIs of your prediction instances.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "[your-bucket-name]"

if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = f"automl-text-notebook-{TIMESTAMP}"

BUCKET_URI = f"gs://{BUCKET_NAME}"

! gsutil mb -l $REGION $BUCKET_URI

In [ ]:
from google.cloud import storage

# Instantiate the Storage client and create the new bucket
storage = storage.Client()
bucket = storage.bucket(BUCKET_NAME)

# Iterate over the prediction instances, creating a new TXT file
# for each.
input_file_data = []
for count, instance in enumerate(instances):
    instance_name = f"input_{count}.txt"
    instance_file_uri = f"{BUCKET_URI}/{instance_name}"

    # Add the data to store in the JSONL input file.
    tmp_data = {"content": instance_file_uri, "mimeType": "text/plain"}
    input_file_data.append(tmp_data)

    # Create the new instance file
    blob = bucket.blob(instance_name)
    blob.upload_from_string(instance)

input_str = "\n".join([str(d) for d in input_file_data])
file_blob = bucket.blob(f"{input_file_name}")
file_blob.upload_from_string(input_str)

Now that you have the bucket with the prediction instances ready, you can send a batch prediction request to Vertex AI. When you send a request to the service, you must provide the URI of your JSONL file and your output bucket, including the `gs://` protocols.

With the Python SDK, you can create a batch prediction job by calling `Model.batch_predict()`.

In [ ]:
job_display_name = "e2e-text-classification-batch-prediction-job"
model = aiplatform.Model(model_name=model_name)

batch_prediction_job = model.batch_predict(
    job_display_name=job_display_name,
    gcs_source=f"{BUCKET_URI}/{input_file_name}",
    gcs_destination_prefix=f"{BUCKET_URI}/output",
    sync=True,
)

batch_prediction_job_name = batch_prediction_job.resource_name

Once the batch prediction job completes, the Python SDK prints out the resource name of the batch prediction job in the format `projects/[PROJECT_ID]/locations/[LOCATION]/batchPredictionJobs/[BATCH_PREDICTION_JOB_ID]`. You can query the Vertex AI service for the status of the batch prediction job using its ID.

The following code snippet demonstrates how to create an instance of the `BatchPredictionJob` class to review its status. Note that you need the full resource name printed out from the Python SDK for this snippet.


In [ ]:
from google.cloud.aiplatform import jobs

job = jobs.BatchPredictionJob(batch_prediction_job_name)
print(f"Batch prediction job state: {str(job.state)}")

After the batch job has completed, you can view the results of the job in your output Storage bucket. You might want to first list all of the files in your output bucket to find the URI of the output file.

In [ ]:
BUCKET_OUTPUT = f"{BUCKET_URI}/output"

! gsutil ls -a $BUCKET_OUTPUT

The output from the batch prediction job should be contained in a folder ("prefix") that includes the name of the batch prediction job plus a time stamp for when it was created.

For example, if your batch prediction job name is `my-job` and your bucket name is `my-bucket`, the URI of the folder containing your output might look like the following:

```
gs://my-bucket/output/prediction-my-job-2021-06-04T19:54:25.889262Z/
```

Next, you can list the contents of the folder containing the output--including the prefix--to get the full URI of the output file. The output is contained in a JSONL file within the folder.

In [ ]:
output_folder_name = "[your-batch-prediction-output-folder]"
BUCKET_OUTPUT_FOLDER = f"gs://{BUCKET_NAME}}/output/{output_folder_name}"

! gsutil ls $BUCKET_OUTPUT_FOLDER

With the full URI of the output file printed out by the previous command, you can download the JSONL output file and read the results. In this tutorial, you use the [`jsonlines`](https://jsonlines.readthedocs.io/en/latest/) library to read the output results.

The following cell opens up the JSONL output file and then prints the predictions for each instance.

In [ ]:
FILE_NAME = "predictions_00001.jsonl"
OUTPUT_FILE_URI = f"{BUCKET_OUTPUT_FOLDER}/{FILE_NAME}"

! gsutil cp $OUTPUT_FILE_URI ./data

import jsonlines

with jsonlines.open(f"data/{FILE_NAME}") as reader:
    for result in reader.iter(type=dict, skip_invalid=True):
        instance = result["instance"]
        prediction = result["prediction"]
        print(f"\ninstance: {instance['content']}")
        for key, output in prediction.items():
            print(f"\n{key}: {output}")

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

* Dataset
* Training job
* Model
* Endpoint
* Batch prediction
* Batch prediction bucket

In [ ]:
delete_training_job = True
delete_model = True

! gsutil rm -r $BUCKET_URI

batch_prediction_job.delete()

endpoint.delete()

model.delete()

text_dataset.delete()

# Training job
job.delete()

## Next Steps

After completing this tutorial, see the following documentation pages to learn more about Vertex AI:

* [Preparing text training data](https://cloud.google.com/vertex-ai/docs/datasets/prepare-text)
* [Training an AutoML model using the API](https://cloud.google.com/vertex-ai/docs/training/automl-api#text)
* [Evaluating AutoML models](https://cloud.google.com/vertex-ai/docs/training/evaluating-automl-models#text)
* [Deploying a model using ther Vertex AI API](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api#aiplatform_create_endpoint_sample-python)
* [Getting online predictions from AutoML models](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api#aiplatform_create_endpoint_sample-python)
* [Getting batch predictions](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions#text)